<h1 align='center'>Welcome to my Notebook</h1>

In [47]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold,KFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_ty_PI_s41.csv', index_col=0)
#X = csl_df.drop('high_EBL', axis=1, inplace=False)
#y = csl_df['high_EBL'].values
#X = csl_df.drop('trans_loss', axis=1, inplace=False)
#y = csl_df['trans_loss'].values
X = csl_df.drop('transfus_yes', axis=1, inplace=False)
y = csl_df['transfus_yes'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,
                                                    random_state=7)

<h1 align='center'>Exploratory Data Analysis</h1>

In [3]:
X

,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,...,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI,Sitenum
MomID,,,,,,,,,,,,,,,,,,,,,
41-00002,0,8,8,8,99.0,83,0.0,77,150.0,7,...,0,0,0,1,0,37,33,1,1,41
41-00003,0,6,3,2,2.9,77,70.0,1,105.0,8,...,0,0,0,0,0,36,22,1,1,41
41-00004,0,5,3,2,3.3,65,70.0,1,122.0,8,...,0,0,0,0,0,34,31,1,1,41
41-00005,0,11,2,3,4.0,77,80.0,77,138.0,8,...,0,0,0,0,0,31,21,1,1,41
41-00006,0,11,1,3,5.0,98,100.0,77,134.0,8,...,0,0,0,0,0,21,22,1,1,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41-47001,0,3,3,2,5.7,66,0.0,77,103.0,8,...,1,0,0,1,0,33,18,1,0,41
41-47002,0,4,2,2,4.0,75,70.0,77,127.0,8,...,1,0,0,0,0,30,37,1,1,41
41-47003,0,8,8,8,5.7,78,90.0,77,119.0,7,...,0,0,0,0,0,30,20,1,0,41


In [4]:
csl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42856 entries, 41-00002 to 41-47005
Columns: 196 entries, transfus_yes to Sitenum
dtypes: float64(4), int64(192)
memory usage: 64.4+ MB


In [5]:
X.skew()

Accrete           50.180851
AdmBishop         -0.600826
Admcervpos         1.275821
Admconsistency     1.420288
Admcontract        2.167352
                    ...    
new_age            0.350433
new_BMI            1.525586
new_high_Age      -1.427224
new_high_BMI      -0.649968
Sitenum            0.000000
Length: 195, dtype: float64

In [6]:
#all_features = pd.concat([train.drop(['id','Response'],axis=1),test.drop('id',axis=1)],axis=0)
#y = train['Response']

Let's combine the train and test set so our transformations are easier as we don't have to apply them seperately to each set

<h1 align='center'>Categorical Features Data Analysis</h1>

In [7]:
%precision %.3f
#np.set_printoptions(precision=3)
print(f'{np.bincount(y)[1]/len(y):.4f}')
print(f'{np.bincount(y_train)[1]/len(y_train):.4f}')

0.0913
0.0913


In [1]:
fig1, ax1 = plt.subplots()
ax1.pie(csl_df['transfus_yes'].value_counts(), labels=['< 1L','>= 1L'], autopct='%1.1f%%')
plt.show()

NameError: name 'plt' is not defined

In [10]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)
#plt.show()

In [11]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)

In [2]:
sns.countplot(csl_df['transfus_yes'])
plt.show()

NameError: name 'sns' is not defined

We can see from the above visualisations that:

* The Data is highly imbalanced

In [3]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_age'],csl_df['transfus_yes'])
plt.show()

NameError: name 'plt' is not defined

In [4]:
sns.boxplot(csl_df['new_age'])
plt.show()

NameError: name 'sns' is not defined

In [5]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_BMI'],csl_df['transfus_yes'])
plt.show()

NameError: name 'plt' is not defined

In [6]:
sns.boxplot(csl_df['new_BMI'])
plt.show()

NameError: name 'sns' is not defined

<h1 align='center'>Modelling</h1>

In [19]:
#X = all_features.iloc[:len(train),:]
#X_test = all_features.iloc[len(train):,:]

#kf = StratifiedKFold(n_splits=12,shuffle=True,random_state=42)

We define resplit our train and test set, and set up 12 Fold Stratified Cross Validation. It is important to stratify as this ensures that during our training and validation, we split according to target distribution

In [20]:
#for train_index,val_index in kf.split(X,y):
#    X_train,X_val = X.iloc[train_index],X.iloc[val_index],
#    y_train,y_val = y.iloc[train_index],y.iloc[val_index],

In [21]:
# MWB
print(X_train.shape)
print(X.shape)
print(y.shape)

(29999, 195)
(42856, 195)
(42856,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [22]:
#sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
#X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
X_rus, y_rus = X_train, y_train

In [23]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(29999, 195)
(29999,)
[27261  2738]


<h1 align='center'>Basic LightGBM</h1>

Let's fit a vanilla LGBMClassifier on the undersampled data and evaluate it

In [40]:
from sklearn.metrics import matthews_corrcoef
lgb_rus = LGBMClassifier(class_weight='balanced')   # If not undersampling
# lgb_rus = LGBMClassifier()
lgb_rus.fit(X_rus, y_rus)
y_pred = lgb_rus.predict(X_test)
probs = lgb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC pred: {roc_auc_score(y_test, y_pred)}')
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[7357 4326]
 [ 667  507]]
              precision    recall  f1-score   support

           0       0.92      0.63      0.75     11683
           1       0.10      0.43      0.17      1174

    accuracy                           0.61     12857
   macro avg       0.51      0.53      0.46     12857
weighted avg       0.84      0.61      0.69     12857

ROC AUC pred: 0.5307876468684897
ROC AUC prob1: 0.5633500298414053
MCC: 0.03661962321698308
F2: 0.2660300136425648
PR_AUC: 0.1218587317412832


<h1 align='center'>Hyperparameter Tuning</h1>

In [50]:
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators',100,500)
    num_leaves = trial.suggest_int('num_leaves',10,500)
    max_depth = trial.suggest_int('max_depth',4,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.0001,1)
    min_child_samples = trial.suggest_int('min_child_samples',10,50)
    model = LGBMClassifier(n_estimators=n_estimators, num_leaves=num_leaves, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_child_samples=min_child_samples, class_weight='balanced')
#    model = LGBMClassifier(n_estimators=n_estimators, num_leaves=num_leaves, 
#                           max_depth=max_depth, learning_rate=learning_rate, 
#                           min_child_samples=min_child_samples)
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_rus, y_rus)
    y_pred = model.predict(X_test)
    probs = model.predict_proba(X_test)
    prob1 = probs[:, 1]  # Only positives
    precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
    roc = roc_auc_score(y_test, prob1)
    mcc = matthews_corrcoef(y_test, y_pred)  # MWB
    f2 = fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]
    pr_auc = auc(recall, precision)
    avg_precision = average_precision_score(y_test, prob1)
    print(f'PR_AUC: {pr_auc}')
    print(f'avg_prec: {avg_precision}')
    print(f'MCC: {mcc}')
    return avg_precision

sampler = TPESampler(seed=42)
study = optuna.create_study(sampler=sampler,direction='maximize')
study.optimize(objective,n_trials=60)
#study.optimize(objective,n_trials=10)

[I 2021-12-01 18:34:31,938] A new study created in memory with name: no-name-c990f7d8-fc80-4140-bf34-b24975ccd8b4
[I 2021-12-01 18:34:34,919] Trial 0 finished with value: 0.11005497317773205 and parameters: {'n_estimators': 250, 'num_leaves': 476, 'max_depth': 16, 'learning_rate': 0.5986986183486169, 'min_child_samples': 16}. Best is trial 0 with value: 0.11005497317773205.


PR_AUC: 0.10974259198882269
avg_prec: 0.11005497317773205
MCC: 0.014733785828156872


[I 2021-12-01 18:34:35,726] Trial 1 finished with value: 0.1053955502634013 and parameters: {'n_estimators': 162, 'num_leaves': 38, 'max_depth': 18, 'learning_rate': 0.6011549002420344, 'min_child_samples': 39}. Best is trial 0 with value: 0.11005497317773205.


PR_AUC: 0.10492647555343806
avg_prec: 0.1053955502634013
MCC: 0.01758013610844953


[I 2021-12-01 18:34:38,219] Trial 2 finished with value: 0.11277712042198546 and parameters: {'n_estimators': 108, 'num_leaves': 486, 'max_depth': 18, 'learning_rate': 0.21241787676720833, 'min_child_samples': 17}. Best is trial 2 with value: 0.11277712042198546.


...

PR_AUC: 0.10752234292502102
avg_prec: 0.1078765517881334
MCC: 0.011526809110445336


[I 2021-12-01 18:37:06,981] Trial 58 finished with value: 0.11181962346980692 and parameters: {'n_estimators': 284, 'num_leaves': 291, 'max_depth': 14, 'learning_rate': 0.6516101827772689, 'min_child_samples': 44}. Best is trial 26 with value: 0.12328113612307391.


PR_AUC: 0.11142766929311734
avg_prec: 0.11181962346980692
MCC: 0.017503347519252928


[I 2021-12-01 18:37:08,882] Trial 59 finished with value: 0.10983291591518195 and parameters: {'n_estimators': 185, 'num_leaves': 177, 'max_depth': 18, 'learning_rate': 0.9683274449937719, 'min_child_samples': 38}. Best is trial 26 with value: 0.12328113612307391.


PR_AUC: 0.10949817073110138
avg_prec: 0.10983291591518195
MCC: 0.024178208431398732


In [51]:
lgb_params = study.best_params
lgb_params['random_state'] = 42
lgb_params['class_weight'] = 'balanced'
lgb = LGBMClassifier(**lgb_params)
lgb.fit(X_rus, y_rus)
y_pred = lgb.predict(X_test)
probs = lgb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test,lgb.predict(X_test)))
print(f'ROC AUC pred: {roc_auc_score(y_test, y_pred)}')
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[9424 2259]
 [ 889  285]]
              precision    recall  f1-score   support

           0       0.91      0.81      0.86     11683
           1       0.11      0.24      0.15      1174

    accuracy                           0.76     12857
   macro avg       0.51      0.52      0.51     12857
weighted avg       0.84      0.76      0.79     12857

ROC AUC pred: 0.5247009625803505
ROC AUC prob1: 0.5664815911411053
MCC: 0.03571939117404768
F2: 0.1968232044198895
PR_AUC: 0.12295267082949518


In [44]:
lgb_params = study.best_params
lgb_params['random_state'] = 42
lgb_params['class_weight'] = 'balanced'
lgb = LGBMClassifier(**lgb_params)
lgb.fit(X_rus, y_rus)
y_pred = lgb.predict(X_test)
probs = lgb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test,lgb.predict(X_test)))
print(f'ROC AUC pred: {roc_auc_score(y_test, y_pred)}')
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[2515 9168]
 [ 124 1050]]
              precision    recall  f1-score   support

           0       0.95      0.22      0.35     11683
           1       0.10      0.89      0.18      1174

    accuracy                           0.28     12857
   macro avg       0.53      0.55      0.27     12857
weighted avg       0.88      0.28      0.34     12857

ROC AUC Score: 0.554824122354282
ROC AUC pred: 0.554824122354282
ROC AUC prob1: 0.5799649048159055
MCC: 0.07820063439305325
F2: 0.35201823789727776
PR_AUC: 0.13689131910586008


In [65]:
lgb_params

{'n_estimators': 214,
 'num_leaves': 426,
 'max_depth': 20,
 'learning_rate': 0.011,
 'min_child_samples': 44,
 'random_state': 42}

<h1 align='center'>Thanks and make sure to learn!</h1>